<a href="https://colab.research.google.com/github/Kvillalobos210/Repos/blob/master/Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from collections import deque
from openpyxl import load_workbook
import random

# Función para leer los datos del archivo Excel
def leer_datos_excel(ruta_archivo):
    # Cargar el libro de trabajo y las hojas
    libro = load_workbook(filename=ruta_archivo)
    hoja_pref = libro['Caso 1_Turnos']
    hoja_libres = libro['Caso 1_DiasLibres']
    hoja_turnos = libro['Caso 1_EnfermerasPorTurno']

    # Leer las preferencias de las enfermeras
    preferencias = pd.DataFrame(hoja_pref.values)
    preferencias.columns = preferencias.iloc[0]
    preferencias = preferencias.drop(0)
    preferencias = preferencias.set_index('Enfermera').T.to_dict('list')

    # Leer los días libres de las enfermeras
    dias_libres = pd.DataFrame(hoja_libres.values)
    dias_libres.columns = dias_libres.iloc[0]
    dias_libres = dias_libres.drop(0)
    dias_libres = dias_libres.set_index('Enfermera').to_dict()['Dia Libre']

    # Leer la cantidad de enfermeras por turno
    enfermeras_por_turno = pd.DataFrame(hoja_turnos.values)
    enfermeras_por_turno = enfermeras_por_turno.iloc[1, 0:].tolist()

    return preferencias, dias_libres, enfermeras_por_turno

# Función para generar la asignación de turnos usando Round Robin
def generar_asignacion_turnos_round_robin(preferencias, dias_libres, enfermeras_por_turno):
    num_dias = len(next(iter(preferencias.values())))
    num_turnos_por_dia = len(enfermeras_por_turno)
    asignacion_por_enfermera = {enfermera: [None] * num_dias for enfermera in preferencias}  # Diccionario para la asignación por enfermera

    cola_enfermeras = deque(preferencias.keys())

    # Asignar los días libres primero
    for enfermera, dia_libre in dias_libres.items():
        asignacion_por_enfermera[enfermera][dia_libre - 1] = 0  # Los días se cuentan desde 1 en el input

    for dia in range(num_dias):
        enfermeras_disponibles = set(preferencias.keys()) - set(enfermera for enfermera, dia_libre in dias_libres.items() if dia_libre-1 == dia)  # Excluir enfermeras con día libre
        for turno in range(num_turnos_por_dia):
            turno_actual = turno + 1
            enfermeras_turno = []

            # Priorizar las preferencias
            for enfermera, dias in preferencias.items():
                if dias[dia] == turno_actual and enfermera in enfermeras_disponibles and len(enfermeras_turno) < enfermeras_por_turno[turno]:
                    enfermeras_turno.append(enfermera)
                    enfermeras_disponibles.remove(enfermera)  # La enfermera ya no está disponible para otros turnos ese día

            # Completar con Round Robin si es necesario
            while len(enfermeras_turno) < enfermeras_por_turno[turno] and enfermeras_disponibles:
                enfermera_actual = cola_enfermeras[0]
                if enfermera_actual in enfermeras_disponibles:
                    enfermeras_turno.append(enfermera_actual)
                    enfermeras_disponibles.remove(enfermera_actual)
                cola_enfermeras.rotate(-1)

            # Asignar el turno a las enfermeras seleccionadas y actualizar la asignación por enfermera
            for enfermera in enfermeras_turno:
                asignacion_por_enfermera[enfermera][dia] = turno_actual

    # Asegurar que no hay None en la asignación final, si hay, asignar un turno disponible
    for enfermera, turnos in asignacion_por_enfermera.items():
        for dia in range(num_dias):
            if turnos[dia] is None:  # Si hay un turno no asignado, asignar un turno disponible
                # Encontrar los turnos ya ocupados en ese día
                turnos_ocupados = {asignacion_por_enfermera[e][dia] for e in preferencias if asignacion_por_enfermera[e][dia] is not None}
                # Encontrar turnos disponibles
                turnos_disponibles = set(range(1, num_turnos_por_dia + 1)) - turnos_ocupados
                if turnos_disponibles:
                    turnos[dia] = turnos_disponibles.pop()
                else:
                    # Si no hay turnos disponibles, asignar el primer turno que no sea el día libre
                    for turno_potencial in range(1, num_turnos_por_dia + 1):
                        if turno_potencial != turnos[dia_libre - 1]:
                            turnos[dia] = turno_potencial
                            break

    return asignacion_por_enfermera

# Generar la asignación de turnos con la función modificada
preferencias, dias_libres, enfermeras_por_turno = leer_datos_excel('Datasets_Enfermeras_Generados_v6.xlsx')
numero_total_enfermeras = len(preferencias)
numero_de_turno_por_dia = len(enfermeras_por_turno)
num_dias_total = num_dias = len(next(iter(preferencias.values())))
asignacion_por_enfermera = generar_asignacion_turnos_round_robin(preferencias, dias_libres, enfermeras_por_turno)

asignacion_por_enfermera

{1: [2, 1, 0, 3, 2, 2, 2],
 2: [2, 3, 1, 3, 0, 1, 3],
 3: [1, 3, 2, 0, 2, 1, 1],
 4: [2, 3, 1, 2, 1, 0, 3],
 5: [3, 1, 1, 0, 3, 2, 2],
 6: [2, 2, 0, 3, 2, 3, 3],
 7: [3, 1, 2, 0, 2, 1, 1],
 8: [1, 1, 2, 1, 1, 3, 0],
 9: [1, 1, 0, 2, 3, 2, 2],
 10: [0, 1, 2, 1, 1, 2, 2],
 11: [3, 2, 1, 1, 1, 0, 1],
 12: [0, 2, 1, 1, 1, 3, 1],
 13: [0, 2, 3, 2, 1, 1, 2],
 14: [1, 2, 0, 1, 1, 2, 2],
 15: [0, 2, 3, 2, 1, 1, 1],
 16: [1, 2, 1, 1, 3, 0, 1],
 17: [1, 1, 3, 2, 0, 1, 1]}

In [ ]:
import pandas as pd
from collections import deque
from openpyxl import load_workbook
import random

# Función para leer los datos del archivo Excel
def leer_datos_excel(ruta_archivo):
    # Cargar el libro de trabajo y las hojas
    libro = load_workbook(filename=ruta_archivo)
    hoja_pref = libro['Caso 1_Turnos']
    hoja_libres = libro['Caso 1_DiasLibres']
    hoja_turnos = libro['Caso 1_EnfermerasPorTurno']

    # Leer las preferencias de las enfermeras
    preferencias = pd.DataFrame(hoja_pref.values)
    preferencias.columns = preferencias.iloc[0]
    preferencias = preferencias.drop(0)
    preferencias = preferencias.set_index('Enfermera').T.to_dict('list')

    # Leer los días libres de las enfermeras
    dias_libres = pd.DataFrame(hoja_libres.values)
    dias_libres.columns = dias_libres.iloc[0]
    dias_libres = dias_libres.drop(0)
    dias_libres = dias_libres.set_index('Enfermera').to_dict()['Dia Libre']

    # Leer la cantidad de enfermeras por turno
    enfermeras_por_turno = pd.DataFrame(hoja_turnos.values)
    enfermeras_por_turno = enfermeras_por_turno.iloc[1, 0:].tolist()

    return preferencias, dias_libres, enfermeras_por_turno

# Función para generar la asignación de turnos usando Round Robin
def generar_asignacion_turnos_round_robin(preferencias, dias_libres, enfermeras_por_turno):
    num_dias = len(next(iter(preferencias.values())))
    num_turnos_por_dia = len(enfermeras_por_turno)
    asignacion_por_enfermera = {enfermera: [None] * num_dias for enfermera in preferencias}  # Diccionario para la asignación por enfermera

    cola_enfermeras = deque(preferencias.keys())

    # Asignar los días libres primero
    for enfermera, dia_libre in dias_libres.items():
        asignacion_por_enfermera[enfermera][dia_libre - 1] = 0  # Los días se cuentan desde 1 en el input

    for dia in range(num_dias):
        enfermeras_disponibles = set(preferencias.keys()) - set(enfermera for enfermera, dia_libre in dias_libres.items() if dia_libre-1 == dia)  # Excluir enfermeras con día libre
        for turno in range(num_turnos_por_dia):
            turno_actual = turno + 1
            enfermeras_turno = []

            # Priorizar las preferencias
            for enfermera, dias in preferencias.items():
                if dias[dia] == turno_actual and enfermera in enfermeras_disponibles and len(enfermeras_turno) < enfermeras_por_turno[turno]:
                    enfermeras_turno.append(enfermera)
                    enfermeras_disponibles.remove(enfermera)  # La enfermera ya no está disponible para otros turnos ese día

            # Completar con Round Robin si es necesario
            while len(enfermeras_turno) < enfermeras_por_turno[turno] and enfermeras_disponibles:
                enfermera_actual = cola_enfermeras[0]
                if enfermera_actual in enfermeras_disponibles:
                    enfermeras_turno.append(enfermera_actual)
                    enfermeras_disponibles.remove(enfermera_actual)
                cola_enfermeras.rotate(-1)

            # Asignar el turno a las enfermeras seleccionadas y actualizar la asignación por enfermera
            for enfermera in enfermeras_turno:
                asignacion_por_enfermera[enfermera][dia] = turno_actual

    # Asegurar que no hay None en la asignación final, si hay, asignar un turno disponible
    for enfermera, turnos in asignacion_por_enfermera.items():
        for dia in range(num_dias):
            if turnos[dia] is None:  # Si hay un turno no asignado, asignar un turno disponible
                # Encontrar los turnos ya ocupados en ese día
                turnos_ocupados = {asignacion_por_enfermera[e][dia] for e in preferencias if asignacion_por_enfermera[e][dia] is not None}
                # Encontrar turnos disponibles
                turnos_disponibles = set(range(1, num_turnos_por_dia + 1)) - turnos_ocupados
                if turnos_disponibles:
                    turnos[dia] = turnos_disponibles.pop()
                else:
                    # Si no hay turnos disponibles, asignar el primer turno que no sea el día libre
                    for turno_potencial in range(1, num_turnos_por_dia + 1):
                        if turno_potencial != turnos[dia_libre - 1]:
                            turnos[dia] = turno_potencial
                            break

    return asignacion_por_enfermera

# Generar la asignación de turnos con la función modificada
preferencias, dias_libres, enfermeras_por_turno = leer_datos_excel('Datasets_Enfermeras_Generados_v6.xlsx')
numero_total_enfermeras = len(preferencias)
numero_de_turno_por_dia = len(enfermeras_por_turno)
num_dias_total = num_dias = len(next(iter(preferencias.values())))
asignacion_por_enfermera = generar_asignacion_turnos_round_robin(preferencias, dias_libres, enfermeras_por_turno)


import pandas as pd
from collections import deque
from openpyxl import load_workbook
import random

# Función para leer los datos del archivo Excel
def leer_datos_excel(ruta_archivo):
    # Cargar el libro de trabajo y las hojas
    libro = load_workbook(filename=ruta_archivo)
    hoja_pref = libro['Caso 1_Turnos']
    hoja_libres = libro['Caso 1_DiasLibres']
    hoja_turnos = libro['Caso 1_EnfermerasPorTurno']

    # Leer las preferencias de las enfermeras
    preferencias = pd.DataFrame(hoja_pref.values)
    preferencias.columns = preferencias.iloc[0]
    preferencias = preferencias.drop(0)
    preferencias = preferencias.set_index('Enfermera').T.to_dict('list')

    # Leer los días libres de las enfermeras
    dias_libres = pd.DataFrame(hoja_libres.values)
    dias_libres.columns = dias_libres.iloc[0]
    dias_libres = dias_libres.drop(0)
    dias_libres = dias_libres.set_index('Enfermera').to_dict()['Dia Libre']

    # Leer la cantidad de enfermeras por turno
    enfermeras_por_turno = pd.DataFrame(hoja_turnos.values)
    enfermeras_por_turno = enfermeras_por_turno.iloc[1, 0:].tolist()

    return preferencias, dias_libres, enfermeras_por_turno

# Función para generar la asignación de turnos usando Round Robin
def generar_asignacion_turnos_round_robin(preferencias, dias_libres, enfermeras_por_turno):
    num_dias = len(next(iter(preferencias.values())))
    num_turnos_por_dia = len(enfermeras_por_turno)
    asignacion_por_enfermera = {enfermera: [None] * num_dias for enfermera in preferencias}  # Diccionario para la asignación por enfermera

    cola_enfermeras = deque(preferencias.keys())

    # Asignar los días libres primero
    for enfermera, dia_libre in dias_libres.items():
        asignacion_por_enfermera[enfermera][dia_libre - 1] = 0  # Los días se cuentan desde 1 en el input

    for dia in range(num_dias):
        enfermeras_disponibles = set(preferencias.keys()) - set(enfermera for enfermera, dia_libre in dias_libres.items() if dia_libre-1 == dia)  # Excluir enfermeras con día libre
        for turno in range(num_turnos_por_dia):
            turno_actual = turno + 1
            enfermeras_turno = []

            # Priorizar las preferencias
            for enfermera, dias in preferencias.items():
                if dias[dia] == turno_actual and enfermera in enfermeras_disponibles and len(enfermeras_turno) < enfermeras_por_turno[turno]:
                    enfermeras_turno.append(enfermera)
                    enfermeras_disponibles.remove(enfermera)  # La enfermera ya no está disponible para otros turnos ese día

            # Completar con Round Robin si es necesario
            while len(enfermeras_turno) < enfermeras_por_turno[turno] and enfermeras_disponibles:
                enfermera_actual = cola_enfermeras[0]
                if enfermera_actual in enfermeras_disponibles:
                    enfermeras_turno.append(enfermera_actual)
                    enfermeras_disponibles.remove(enfermera_actual)
                cola_enfermeras.rotate(-1)

            # Asignar el turno a las enfermeras seleccionadas y actualizar la asignación por enfermera
            for enfermera in enfermeras_turno:
                asignacion_por_enfermera[enfermera][dia] = turno_actual

    # Asegurar que no hay None en la asignación final, si hay, asignar un turno disponible
    for enfermera, turnos in asignacion_por_enfermera.items():
        for dia in range(num_dias):
            if turnos[dia] is None:  # Si hay un turno no asignado, asignar un turno disponible
                # Encontrar los turnos ya ocupados en ese día
                turnos_ocupados = {asignacion_por_enfermera[e][dia] for e in preferencias if asignacion_por_enfermera[e][dia] is not None}
                # Encontrar turnos disponibles
                turnos_disponibles = set(range(1, num_turnos_por_dia + 1)) - turnos_ocupados
                if turnos_disponibles:
                    turnos[dia] = turnos_disponibles.pop()
                else:
                    # Si no hay turnos disponibles, asignar el primer turno que no sea el día libre
                    for turno_potencial in range(1, num_turnos_por_dia + 1):
                        if turno_potencial != turnos[dia_libre - 1]:
                            turnos[dia] = turno_potencial
                            break

    return asignacion_por_enfermera

# Generar la asignación de turnos con la función modificada
preferencias, dias_libres, enfermeras_por_turno = leer_datos_excel('Datasets_Enfermeras_Generados_v6.xlsx')
numero_total_enfermeras = len(preferencias)
numero_de_turno_por_dia = len(enfermeras_por_turno)
num_dias_total = num_dias = len(next(iter(preferencias.values())))
asignacion_por_enfermera = generar_asignacion_turnos_round_robin(preferencias, dias_libres, enfermeras_por_turno)


def generar_asignacion_turnos(num_dias, num_turnos_por_dia, num_enfermeras_por_turno, num_enfermeras):
    # Crear un arreglo tridimensional para la asignación de turnos
    asignacion = [[[0 for _ in range(num_enfermeras_por_turno[turno])] for turno in range(num_turnos_por_dia)] for _ in range(num_dias)]
    # Generar la asignación de turnos
    for dia in range(num_dias):
        for turno in range(num_turnos_por_dia):
            # Evitar que una enfermera haga dos turnos seguidos o en un mismo día
            enfermeras_disponibles = list(range(1, num_enfermeras + 1))
            if dia > 0 and turno == 0:
                # Retirar las enfermeras que trabajaron en el último turno del día anterior
                enfermeras_disponibles = [enf for enf in enfermeras_disponibles if enf not in asignacion[dia - 1][-1]]
            for turno_previo in range(turno):
                # Retirar las enfermeras que ya están asignadas en turnos previos del mismo día
                enfermeras_disponibles = [enf for enf in enfermeras_disponibles if enf not in asignacion[dia][turno_previo]]

            # Asignar enfermeras al turno actual

    return asignacion

# Parámetros del problema
num_enfermeras = numero_total_enfermeras
num_dias = 7
num_turnos_por_dia = 3
num_enfermeras_por_turno = enfermeras_por_turno  # Número de enfermeras por turno
# Ejemplo de preferencias de días libres de las enfermeras
preferencias_dias_libres = dias_libres


# Función de aptitud

def calcular_fitness(asignacion, preferencias_dias_libres):
    fitness = 0  # Inicializar el valor de fitness en 0

    for enfermera, dia_preferido in preferencias_dias_libres.items():
        for dia in range(len(asignacion)):
            if dia + 1 == dia_preferido:
                # Si la enfermera tiene el día preferido libre, aumentar el fitness
                turno_del_dia = asignacion[dia][0]
                if enfermera in turno_del_dia:
                    fitness += 1
                turno_del_dia = asignacion[dia][1]
                if enfermera in turno_del_dia:
                    fitness += 1
                turno_del_dia = asignacion[dia][2]
                if enfermera in turno_del_dia:
                    fitness += 1

    return fitness

def tiene_elemento_repetido(lista, elemento):
    return lista.count(elemento) > 1

def reparar_cromosoma(individuo, num_enfermeras):

    for dia in range(len(individuo)):
        for turno in range(len(individuo[dia])):
            for enfermera in range(len(individuo[dia][turno])):
                # Restricción r1: Verificar si la enfermera se repite en el mismo turno
                while tiene_elemento_repetido(individuo[dia][turno], individuo[dia][turno][enfermera]):
                    individuo[dia][turno][enfermera] = random.choice(range(1, num_enfermeras + 1))

                # Restricción r2: Verificar si la enfermera trabajó en el último turno del día anterior
                if dia != 0 and turno == 0:
                    while individuo[dia][turno][enfermera] in individuo[dia - 1][2]:
                        individuo[dia][turno][enfermera] = random.choice(range(1, num_enfermeras + 1))

                # Restricción r3: Verificar si la enfermera está ocupada en los turnos anteriores del mismo día
                if turno > 0:
                    ocupada_en_turnos_anteriores = any(individuo[dia][turno][enfermera] in individuo[dia][t] for t in range(turno))
                    while ocupada_en_turnos_anteriores:
                        individuo[dia][turno][enfermera] = random.choice(range(1, num_enfermeras + 1))
                        ocupada_en_turnos_anteriores = any(individuo[dia][turno][enfermera] in individuo[dia][t] for t in range(turno))

# Operadores genéticos
def seleccion(poblacion, aptitudes):
    # Seleccionar individuos basados en su aptitud
    seleccionados = random.choices(poblacion, weights=[1.0 / (aptitud + 1) for aptitud in aptitudes], k=len(poblacion))
    return seleccionados

def cruzar_padres(padre1, padre2, num_enfermeras_por_turno, num_enfermeras):
    punto_de_corte = random.randint(0, len(padre1) - 1)
    descendiente1 = padre1[:punto_de_corte] + padre2[punto_de_corte:]
    descendiente2 = padre2[:punto_de_corte] + padre1[punto_de_corte:]
    reparar_cromosoma(descendiente1, num_enfermeras)
    reparar_cromosoma(descendiente2, num_enfermeras)
    return descendiente1, descendiente2

def mutar_cromosoma(cromosoma, num_enfermeras_por_turno, num_enfermeras):
    dia = random.randint(0, len(cromosoma) - 1)
    turno = random.randint(0, len(cromosoma[dia]) - 1)
    # Asegurarse de que la mutación cumple con el número de enfermeras requerido para el turno
    enfermeras_disponibles = list(range(1, num_enfermeras + 1))
    nueva_enfermera = random.choice(enfermeras_disponibles)
    cromosoma[dia][turno][0] = nueva_enfermera
    reparar_cromosoma(cromosoma, num_enfermeras)
    return cromosoma

# Algoritmo genético
def algoritmo_genetico(num_generaciones, tamano_poblacion, num_enfermeras_por_turno, num_enfermeras):
    poblacion_inicial = [generar_asignacion_turnos(num_dias, num_turnos_por_dia, num_enfermeras_por_turno, num_enfermeras) for _ in range(tamano_poblacion)]
    los_mejores = []

    for generacion in range(num_generaciones):
        aptitudes = [calcular_fitness(individuo, preferencias_dias_libres) for individuo in poblacion_inicial]
        padres = seleccion(poblacion_inicial, aptitudes)
        descendencia = []

        for i in range(0, len(padres), 2):
            hijo1, hijo2 = cruzar_padres(padres[i], padres[i + 1], num_enfermeras_por_turno, num_enfermeras)
            descendencia.extend([hijo1, hijo2])

        for individuo in descendencia:
            mutar_cromosoma(individuo, num_enfermeras_por_turno, num_enfermeras)

        poblacion_inicial = descendencia
        mejor_individuo = poblacion_inicial[aptitudes.index(min(aptitudes))]
        los_mejores.append(mejor_individuo)

    aptitudes = [calcular_fitness(individuo, preferencias_dias_libres) for individuo in los_mejores]
    mejor_individuo = los_mejores[aptitudes.index(min(aptitudes))]
    return mejor_individuo

# Parámetros del algoritmo genético
num_generaciones = 700
tamano_poblacion = 50
tasa_mutacion = 0.1

mejor_solucion = algoritmo_genetico(num_generaciones, tamano_poblacion, num_enfermeras_por_turno, num_enfermeras)

print(calcular_fitness(mejor_solucion,preferencias_dias_libres))

for dia in range(num_dias):
        for turno in range(num_turnos_por_dia):
            print(f'Día {dia + 1}, Turno {turno + 1}: Enfermeras asignadas - {mejor_solucion[dia][turno]}')


In [ ]:
import pandas as pd
from collections import deque
from openpyxl import load_workbook
import random

# Función para leer los datos del archivo Excel
def leer_datos_excel(ruta_archivo):
    # Cargar el libro de trabajo y las hojas
    libro = load_workbook(filename=ruta_archivo)
    hoja_pref = libro['Caso 1_Turnos']
    hoja_libres = libro['Caso 1_DiasLibres']
    hoja_turnos = libro['Caso 1_EnfermerasPorTurno']

    # Leer las preferencias de las enfermeras
    preferencias = pd.DataFrame(hoja_pref.values)
    preferencias.columns = preferencias.iloc[0]
    preferencias = preferencias.drop(0)
    preferencias = preferencias.set_index('Enfermera').T.to_dict('list')

    # Leer los días libres de las enfermeras
    dias_libres = pd.DataFrame(hoja_libres.values)
    dias_libres.columns = dias_libres.iloc[0]
    dias_libres = dias_libres.drop(0)
    dias_libres = dias_libres.set_index('Enfermera').to_dict()['Dia Libre']

    # Leer la cantidad de enfermeras por turno
    enfermeras_por_turno = pd.DataFrame(hoja_turnos.values)
    enfermeras_por_turno = enfermeras_por_turno.iloc[1, 0:].tolist()

    return preferencias, dias_libres, enfermeras_por_turno


def generar_asignacion_turnos(num_dias, num_turnos_por_dia, num_enfermeras_por_turno, num_enfermeras):
    # Crear un arreglo tridimensional para la asignación de turnos
    asignacion = [[[0 for _ in range(num_enfermeras_por_turno[turno])] for turno in range(num_turnos_por_dia)] for _ in range(num_dias)]
    # Generar la asignación de turnos
    for dia in range(num_dias):
        for turno in range(num_turnos_por_dia):
            # Evitar que una enfermera haga dos turnos seguidos o en un mismo día
            enfermeras_disponibles = list(range(1, num_enfermeras + 1))
            if dia > 0 and turno == 0:
                # Retirar las enfermeras que trabajaron en el último turno del día anterior
                enfermeras_disponibles = [enf for enf in enfermeras_disponibles if enf not in asignacion[dia - 1][-1]]
            for turno_previo in range(turno):
                # Retirar las enfermeras que ya están asignadas en turnos previos del mismo día
                enfermeras_disponibles = [enf for enf in enfermeras_disponibles if enf not in asignacion[dia][turno_previo]]

            # Asignar enfermeras al turno actual
            asignacion[dia][turno] = random.sample(enfermeras_disponibles, num_enfermeras_por_turno[turno])

    return asignacion

# Parámetros del problema
preferencias, dias_libres, enfermeras_por_turno = leer_datos_excel('Datasets_Enfermeras_Generados_v6.xlsx')
num_enfermeras = len(preferencias)
num_dias = 7
num_turnos_por_dia = 3
num_enfermeras_por_turno = enfermeras_por_turno  # Número de enfermeras por turno
# Ejemplo de preferencias de días libres de las enfermeras
preferencias_dias_libres = dias_libres


# Función de aptitud

def calcular_fitness(asignacion, preferencias_dias_libres):
    fitness = 0  # Inicializar el valor de fitness en 0

    for enfermera, dia_preferido in preferencias_dias_libres.items():
        for dia in range(len(asignacion)):
            if dia + 1 == dia_preferido:
                # Si la enfermera tiene el día preferido libre, aumentar el fitness
                turno_del_dia = asignacion[dia][0]
                if enfermera in turno_del_dia:
                    fitness += 1
                turno_del_dia = asignacion[dia][1]
                if enfermera in turno_del_dia:
                    fitness += 1
                turno_del_dia = asignacion[dia][2]
                if enfermera in turno_del_dia:
                    fitness += 1

    return fitness

def tiene_elemento_repetido(lista, elemento):
    return lista.count(elemento) > 1

def reparar_cromosoma(individuo, num_enfermeras):

    for dia in range(len(individuo)):
        for turno in range(len(individuo[dia])):
            for enfermera in range(len(individuo[dia][turno])):
                # Restricción r1: Verificar si la enfermera se repite en el mismo turno
                while tiene_elemento_repetido(individuo[dia][turno], individuo[dia][turno][enfermera]):
                    individuo[dia][turno][enfermera] = random.choice(range(1, num_enfermeras + 1))

                # Restricción r2: Verificar si la enfermera trabajó en el último turno del día anterior
                if dia != 0 and turno == 0:
                    while individuo[dia][turno][enfermera] in individuo[dia - 1][2]:
                        individuo[dia][turno][enfermera] = random.choice(range(1, num_enfermeras + 1))

                # Restricción r3: Verificar si la enfermera está ocupada en los turnos anteriores del mismo día
                if turno > 0:
                    ocupada_en_turnos_anteriores = any(individuo[dia][turno][enfermera] in individuo[dia][t] for t in range(turno))
                    while ocupada_en_turnos_anteriores:
                        individuo[dia][turno][enfermera] = random.choice(range(1, num_enfermeras + 1))
                        ocupada_en_turnos_anteriores = any(individuo[dia][turno][enfermera] in individuo[dia][t] for t in range(turno))

# Operadores genéticos
def seleccion(poblacion, aptitudes):
    # Seleccionar individuos basados en su aptitud
    seleccionados = random.choices(poblacion, weights=[1.0 / (aptitud + 1) for aptitud in aptitudes], k=len(poblacion))
    return seleccionados

def cruzar_padres(padre1, padre2, num_enfermeras_por_turno, num_enfermeras):
    punto_de_corte = random.randint(0, len(padre1) - 1)
    descendiente1 = padre1[:punto_de_corte] + padre2[punto_de_corte:]
    descendiente2 = padre2[:punto_de_corte] + padre1[punto_de_corte:]
    reparar_cromosoma(descendiente1, num_enfermeras)
    reparar_cromosoma(descendiente2, num_enfermeras)
    return descendiente1, descendiente2

def mutar_cromosoma(cromosoma, num_enfermeras_por_turno, num_enfermeras):
    dia = random.randint(0, len(cromosoma) - 1)
    turno = random.randint(0, len(cromosoma[dia]) - 1)
    # Asegurarse de que la mutación cumple con el número de enfermeras requerido para el turno
    enfermeras_disponibles = list(range(1, num_enfermeras + 1))
    nueva_enfermera = random.choice(enfermeras_disponibles)
    cromosoma[dia][turno][0] = nueva_enfermera
    reparar_cromosoma(cromosoma, num_enfermeras)
    return cromosoma

# Algoritmo genético
def algoritmo_genetico(num_generaciones, tamano_poblacion, num_enfermeras_por_turno, num_enfermeras):
    poblacion_inicial = [generar_asignacion_turnos(num_dias, num_turnos_por_dia, num_enfermeras_por_turno, num_enfermeras) for _ in range(tamano_poblacion)]
    los_mejores = []

    for generacion in range(num_generaciones):
        aptitudes = [calcular_fitness(individuo, preferencias_dias_libres) for individuo in poblacion_inicial]
        padres = seleccion(poblacion_inicial, aptitudes)
        descendencia = []

        for i in range(0, len(padres), 2):
            hijo1, hijo2 = cruzar_padres(padres[i], padres[i + 1], num_enfermeras_por_turno, num_enfermeras)
            descendencia.extend([hijo1, hijo2])

        for individuo in descendencia:
            mutar_cromosoma(individuo, num_enfermeras_por_turno, num_enfermeras)

        poblacion_inicial = descendencia
        mejor_individuo = poblacion_inicial[aptitudes.index(min(aptitudes))]
        los_mejores.append(mejor_individuo)

    aptitudes = [calcular_fitness(individuo, preferencias_dias_libres) for individuo in los_mejores]
    mejor_individuo = los_mejores[aptitudes.index(min(aptitudes))]
    return mejor_individuo

# Parámetros del algoritmo genético
num_generaciones = 700
tamano_poblacion = 50
tasa_mutacion = 0.1

mejor_solucion = algoritmo_genetico(num_generaciones, tamano_poblacion, num_enfermeras_por_turno, num_enfermeras)

print(calcular_fitness(mejor_solucion,preferencias_dias_libres))

for dia in range(num_dias):
        for turno in range(num_turnos_por_dia):
            print(f'Día {dia + 1}, Turno {turno + 1}: Enfermeras asignadas - {mejor_solucion[dia][turno]}')

10
Día 1, Turno 1: Enfermeras asignadas - [17, 16, 5, 4, 15]
Día 1, Turno 2: Enfermeras asignadas - [11, 14, 7, 12]
Día 1, Turno 3: Enfermeras asignadas - [8, 10, 9]
Día 2, Turno 1: Enfermeras asignadas - [15, 4, 16, 12, 6]
Día 2, Turno 2: Enfermeras asignadas - [14, 2, 1, 9]
Día 2, Turno 3: Enfermeras asignadas - [8, 3, 11]
Día 3, Turno 1: Enfermeras asignadas - [11, 16, 2, 3, 4]
Día 3, Turno 2: Enfermeras asignadas - [15, 6, 8, 14]
Día 3, Turno 3: Enfermeras asignadas - [17, 7, 1]
Día 4, Turno 1: Enfermeras asignadas - [10, 16, 14, 17, 12]
Día 4, Turno 2: Enfermeras asignadas - [3, 4, 1, 2]
Día 4, Turno 3: Enfermeras asignadas - [13, 9, 11]
Día 5, Turno 1: Enfermeras asignadas - [6, 16, 12, 5, 7]
Día 5, Turno 2: Enfermeras asignadas - [4, 13, 15, 14]
Día 5, Turno 3: Enfermeras asignadas - [3, 17, 1]
Día 6, Turno 1: Enfermeras asignadas - [15, 10, 12, 8, 2]
Día 6, Turno 2: Enfermeras asignadas - [5, 3, 4, 17]
Día 6, Turno 3: Enfermeras asignadas - [7, 9, 11]
Día 7, Turno 1: Enfermeras